In [1]:
from Network import ensembleModel
from DB_Manage import DB_Bot
from DataLabeling import DataLabeling
from DataScaler import Data_StandardScaler
from Indicator import DataManage
import keras
import tensorflow as tf

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.1.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
data = DB_Bot("BTC_USDT_30m").GetData()
parameter = [
        {"rsi" : {"period" : 14}},
        {"ma" : {"period" : 7}},
        {"ma" : {"period" : 25}},
        {"ema" :{"period" : 7}},
        {"ema" :{"period" : 25}},
        {"stochastic" : {"n" : 14,"m" : 5,"t" : 5}},
        {"bb" : {"length" : 21,"std" : 2}},
        {"kdj" : {}},
        {"macd" : {"fast_period": 12, "slow_period" : 26}}
]
DataManageBot = DataManage(data, parameter = parameter)
data = DataManageBot.get_data()
data = Data_StandardScaler(data.drop(["datetime"],axis=1))

In [3]:
data.head()

,open,high,low,close,volume,rsi_14,mean_7,mean_25,ema_7,ema_25,...,fast_d,slow_k,slow_d,BBL,BBM,BBU,BBP,K_9_3,D_9_3,J_9_3
0,-1.274480,-1.274199,-1.273354,-1.273669,-0.686487,0.120325,-1.274129,-1.273538,-1.273979,-1.273696,...,-0.323367,-0.323367,-0.515097,-1.269965,-1.269965,-1.269965,-1.269965,-0.325319,-0.619937,0.060653
1,-1.273631,-1.273471,-1.272736,-1.272807,-0.637243,0.517925,-1.273884,-1.273425,-1.273697,-1.273636,...,0.193881,0.193881,-0.272169,-1.269962,-1.269962,-1.269962,-1.269962,0.229661,-0.324306,0.788296
2,-1.272762,-1.273640,-1.272346,-1.273035,-0.691058,0.387046,-1.273601,-1.273317,-1.273542,-1.273599,...,0.435609,0.435609,-0.021420,-1.269880,-1.269880,-1.269880,-1.269880,0.441760,-0.045624,0.885643
3,-1.272996,-1.273557,-1.272158,-1.273071,-0.727150,0.365852,-1.273470,-1.273296,-1.273435,-1.273567,...,0.580753,0.580753,0.199061,-1.269826,-1.269826,-1.269826,-1.269826,0.564097,0.187198,0.863170
4,-1.273042,-1.273477,-1.272630,-1.272461,-0.724352,0.661120,-1.273218,-1.273259,-1.273202,-1.273490,...,0.953427,0.953427,0.482949,-1.269846,-1.269846,-1.269846,-1.269846,0.973253,0.500122,1.296028


In [4]:
Labeler = DataLabeling(data, 20, "close")
Labeler.run()
Labeled_data = Labeler.data
Labeled_data.head()

100%|██████████| 1077/1077 [00:03<00:00, 344.07it/s]


,open,high,low,close,volume,rsi_14,mean_7,mean_25,ema_7,ema_25,...,slow_k,slow_d,BBL,BBM,BBU,BBP,K_9_3,D_9_3,J_9_3,label
0,-1.274480,-1.274199,-1.273354,-1.273669,-0.686487,0.120325,-1.274129,-1.273538,-1.273979,-1.273696,...,-0.323367,-0.515097,-1.269965,-1.269965,-1.269965,-1.269965,-0.325319,-0.619937,0.060653,0
1,-1.273631,-1.273471,-1.272736,-1.272807,-0.637243,0.517925,-1.273884,-1.273425,-1.273697,-1.273636,...,0.193881,-0.272169,-1.269962,-1.269962,-1.269962,-1.269962,0.229661,-0.324306,0.788296,0
2,-1.272762,-1.273640,-1.272346,-1.273035,-0.691058,0.387046,-1.273601,-1.273317,-1.273542,-1.273599,...,0.435609,-0.021420,-1.269880,-1.269880,-1.269880,-1.269880,0.441760,-0.045624,0.885643,0
3,-1.272996,-1.273557,-1.272158,-1.273071,-0.727150,0.365852,-1.273470,-1.273296,-1.273435,-1.273567,...,0.580753,0.199061,-1.269826,-1.269826,-1.269826,-1.269826,0.564097,0.187198,0.863170,0
4,-1.273042,-1.273477,-1.272630,-1.272461,-0.724352,0.661120,-1.273218,-1.273259,-1.273202,-1.273490,...,0.953427,0.482949,-1.269846,-1.269846,-1.269846,-1.269846,0.973253,0.500122,1.296028,0


In [5]:
Labeled_data.label.value_counts()

1    29340
0    26567
Name: label, dtype: int64

In [6]:
test_size = 10000
X,Y = Labeled_data.drop(["label"],axis=1), Labeled_data.label
x_train = X[:-test_size]
y_train = Y[:-test_size]
x_test = X[-test_size:]
y_test = Y[-test_size:]
print(x_train.shape, y_train.shape)

(45907, 21) (45907,)


In [7]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(256,input_shape = (x_train.shape[1],)))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(64,activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(32,activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(1,activation="sigmoid"))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                    metrics=["accuracy"],
                    loss = tf.keras.losses.BinaryCrossentropy())

2023-04-19 21:39:33.238120: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model.fit(x_train,y_train,epochs=10, batch_size=16)

Epoch 1/10
2691/2870 [===========================>..] - ETA: 0s - loss: 0.6255 - accuracy: 0.6612

KeyboardInterrupt: 

In [9]:
pred = model.predict(x_test)
pred

array([[0.6616046],
       [0.5977718],
       [0.5326964],
       ...,
       [0.5741683],
       [0.5068646],
       [0.5480294]], dtype=float32)

In [10]:
result_label = []
cnt = 0

for i in range(len(pred)):
    if pred[i][0] < 0.2:
        result_label.append(0)
    elif pred[i][0] > 0.8:
        result_label.append(1)
    else:
        result_label.append(-1)
        
ck = 0 # 정답 카운트
cnt = 0 # 총 카운트
report_list = []
report_pred = []
for i in range(len(result_label)):
    if result_label[i] != -1:
        if result_label[i] == y_test.iloc[i]:
            ck += 1
        cnt += 1
print("accuracy :",round(ck/cnt * 100,2),"%")

accuracy : 73.23 %


In [11]:
from backtest import backtest
backtestBot = backtest(data, result_label, test_size, 0.002, 0.0008, 100000)
result = backtestBot.basicStrategy()

/Users/yuhyeonseog/졸작 연구/git/Crypteam-2/backend/ML/backtest.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test['label'] = self.result_label
100%|██████████| 10000/10000 [00:05<00:00, 1797.06it/s]


In [12]:
for i in result:
    print(i,":",result[i])

averageNumberSales : 3.0
totalYield : -0.0005577471386142843
win_rate : 0.0
MDD : 1.49
max_buying : 3
NumberTrading : 1


In [ ]:
data.to_csv("debug.csv")

In [ ]:
import pandas as pd
pd.DataFrame(result_label).to_csv("debugresultlabel.csv")

In [ ]:
result_label = []
cnt = 0

for i in range(len(pred)):
    if pred[i][0] < 0.2:
        result_label.append(0)
    elif pred[i][0] > 0.8:
        result_label.append(1)
    else:
        result_label.append(-1)
        
ck = 0 # 정답 카운트
cnt = 0 # 총 카운트
report_list = []
report_pred = []
for i in range(len(result_label)):
    if result_label[i] != -1:
        if result_label[i] == y_test.iloc[i]:
            ck += 1
        cnt += 1
print("accuracy :",round(ck/cnt * 100,2),"%")

NameError: name 'pred' is not defined

In [ ]:
from tqdm import tqdm
class backtest:
    def __init__(self,data, result_label, test_size, set_amount, fee, max_buy):
        self.data = data # 주가 데이터
        self.result_label = result_label # 머신러닝의 예측값
        self.test_size = test_size # 테스트 데이터의 길이
        self.set_amount = set_amount # 한번 매수할 떄 수량
        self.fee = fee # 한번 거래 할 때 수수료
        self.max_buy = max_buy # 최대 누적 매수 횟수

        # 손절형 순환매매에서 사용하는 변수
        self.buy_flag = 0
        self.sell_flag = 0

        # 공통 사용 변수
        self.quantityBuying = 0 # 현재 매수 수량
        self.BuyingList = [] # 현재 매수 했던 가격들의 리스트
        self.totalYield = 0 # 최종 수익률
        self.totalNumberSales = 0 # 총 매도 횟수
        self.win = 0
        self.quantityBuyingList = [] # 매수 수량을 저장하는 리스트
        self.amount = 0 # 현재 가지고 있는 주식 수량
        self.MDDList = [] # MDD를 계산하기 위한 리스트
        self.average_price = 0 # 평단가

        self.trading_history = []
    def initializes(self):
        self.buy_flag = 0
        self.sell_flag = 0

        # 공통 사용 변수
        self.quantityBuying = 0
        self.BuyingList = []
        self.totalYield = 0
        self.totalNumberSales = 0
        self.win = 0
        self.quantityBuyingList = []
        self.amount = 0
        self.MDDList = []
        self.average_price = 0

    def SellInitializes(self,currYield):
        if currYield > 0:
            self.win += 1
        self.average_price = 0
        self.totalNumberSales += 1
        self.totalYield += currYield
        self.amount = 0
        self.BuyingList.clear()
        self.quantityBuying = 0
        self.buy_flag = 0
        self.sell_flag = 0
        

    def basicStrategy(self):
        x_test = self.data.iloc[-self.test_size:]
        x_test['label'] = self.result_label
        self.initializes()
        for i in tqdm(range(len(x_test))):
            
            if x_test.iloc[i]['label'] == 1 and self.quantityBuying < self.max_buy:
                self.quantityBuying += 1
                self.average_price = ( self.average_price * self.amount + x_test.iloc[i]['close'] * self.set_amount) / (self.amount + self.set_amount)
                self.amount += self.set_amount
                self.BuyingList.append(x_test.iloc[i]['close'])

            if x_test.iloc[i]['label'] == 0 and self.quantityBuying != 0:
                currYield = (x_test.iloc[i]['close']-self.average_price) * self.amount - (self.average_price * self.fee * self.amount)
                self.quantityBuyingList.append(self.quantityBuying)

                self.SellInitializes(currYield)

            if self.quantityBuying > 0:
                self.MDDList.append(round((sum(self.BuyingList)/self.quantityBuying-x_test.iloc[i]['close'])/x_test.iloc[i]['close'],2))
        self.quantityBuyingList.sort()
        self.MDDList.sort()
        
        if len(self.quantityBuyingList) == 0 or self.totalNumberSales == 0:
            print("Zero DivisionError")
            raise ZeroDivisionError
        return {
            "averageNumberSales" : sum(self.quantityBuyingList)/len(self.quantityBuyingList),
            "totalYield" : self.totalYield,
            "win_rate" : self.win/self.totalNumberSales,
            "MDD" : self.MDDList[-1],
            "max_buying" : self.quantityBuyingList[-1],
            "NumberTrading" : self.totalNumberSales
        }
    
    def WaitingStrategy(self, term = 5):
        x_test = self.data.iloc[-self.test_size:]
        x_test['label'] = self.result_label
        self.initializes()

        waiting = 0
        for i in tqdm(range(len(x_test))):
            
            if x_test.iloc[i]['label'] == 1 and self.quantityBuying < self.max_buy and waiting > term:
                self.quantityBuying += 1
                self.average_price = ( self.average_price * self.amount + x_test.iloc[i]['close'] * self.set_amount) / (self.amount + self.set_amount)
                self.amount += self.set_amount
                self.BuyingList.append(x_test.iloc[i]['close'])
                self.trading_history.append(["buy", i, None])

            elif x_test.iloc[i]['label'] == 0 and self.quantityBuying != 0:
                currYield = (x_test.iloc[i]['close']-self.average_price) * self.amount - (self.average_price * self.fee * self.amount)
                self.quantityBuyingList.append(self.quantityBuying)

                self.SellInitializes(currYield)
                self.trading_history.append(["sell", i, currYield])
                waiting = 0

            if x_test.iloc[i]["label"] == 1:
                waiting += 1

            if self.quantityBuying > 0:
                self.MDDList.append(round((sum(self.BuyingList)/self.quantityBuying-x_test.iloc[i]['close'])/x_test.iloc[i]['close'],2))
        self.quantityBuyingList.sort()
        self.MDDList.sort()
        
        if len(self.quantityBuyingList) == 0 or self.totalNumberSales == 0:
            print("Zero DivisionError")
            raise ZeroDivisionError
        return {
            "averageNumberSales" : sum(self.quantityBuyingList)/len(self.quantityBuyingList),
            "totalYield" : self.totalYield,
            "win_rate" : self.win/self.totalNumberSales,
            "MDD" : self.MDDList[-1],
            "max_buying" : self.quantityBuyingList[-1],
            "NumberTrading" : self.totalNumberSales
        }
import pandas as pd
from DB_Manage import DB_Bot
data = DB_Bot("BTC_USDT_30m").GetData()
result_label = pd.read_csv("debugresultlabel.csv").to_numpy()[:,1].tolist()
object = backtest(data, result_label, 10000, 0.02, 0.0008, 10000)
print(object.WaitingStrategy(term=10))

/var/folders/5v/ywkv8vl52zs0xz8k0x15y8fm0000gn/T/ipykernel_58663/2578130533.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test['label'] = self.result_label
100%|██████████| 10000/10000 [00:05<00:00, 1902.24it/s]

{'averageNumberSales': 9.285714285714286, 'totalYield': 449.87576815999904, 'win_rate': 0.2857142857142857, 'MDD': 0.16, 'max_buying': 21, 'NumberTrading': 7}


In [ ]:
trading_history = pd.DataFrame(object.trading_history,columns=["type", "index", "sell_price"])
trading_history

,type,index,sell_price
0,buy,1165,NaN
1,sell,1351,-0.373190
2,buy,2074,NaN
3,buy,2076,NaN
4,buy,2170,NaN
...,...,...,...
67,buy,8813,NaN
68,buy,8844,NaN
69,buy,8895,NaN
70,buy,8901,NaN


In [13]:
result

{'averageNumberSales': 3.0,
 'totalYield': -0.0005577471386142843,
 'win_rate': 0.0,
 'MDD': 1.49,
 'max_buying': 3,
 'NumberTrading': 1}

In [14]:
import json
json_string = json.dumps(result)
print(json_string)

{"averageNumberSales": 3.0, "totalYield": -0.0005577471386142843, "win_rate": 0.0, "MDD": 1.49, "max_buying": 3, "NumberTrading": 1}
